# DATA LOAD

Read the original Excel information and transform it to be processed.

The original information is the folder 'data/original', classified by years and in some file by year

In every file exists multiple sheet, some of them with userful information and others that want be used.

The files and sheets will be detected using the cell headers. 

The preprocessed information will be stored in folder 'data/preprocessed', classified by language.


In [1]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
import xlrd
from colorama import Fore

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [3]:
import spacy
from spacy_langdetect import LanguageDetector

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")
# ca_fasttext_wiki
# es_core_news_sm
# en_core_web_sm

nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)



In [4]:
import pycld2 as cld2

In [5]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
debug = True

In [6]:
def delete_preprocessed_files():
    
    languages = ["ca", "es", "en", "xx"]
    for language in languages:
        url = pathdest + "comentaris_" + language + ".csv"
        if os.path.exists(url):
            os.remove(url)
            if debug:
                print ("File deleted: " + url)
    
    

In [7]:
# Test if a file has the correct format to load his information

def validateSheet(data):

    cols = data.columns
    if (cols.size > 5):
        if (('Codi pregunta' in cols) or ('Comentari' in cols) or ('Tipus incidencia' in cols)):
            if (('Codi pregunta' in cols) and ('Comentari' in cols) and ('Tipus incidencia' in cols)):
                print (Fore.GREEN + " " * 8  + "VALID")
                return True
            else:
                print (Fore.RED + " " * 8 + "WARNING (see column names)")
                print (cols)
                return False
        else:
            print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")
            return False
    else:
        print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")
        return False


In [8]:
# Reorganize information to be exported in preprocessed files

def process_file (data_ori, file):
# folder = "17-18"
# file = "assignatura_professor_comentaris_LS_17-18_1s.xlsx"
# sheet = "Hoja1"
# xlsx = pd.ExcelFile(pathori+"/"+folder+"/"+file)
# data_ori = pd.read_excel(xlsx, sheet)

#    print(Fore.BLACK + "Process_file")
#    data_ori = data_ori[data_ori["Comentari"].notna]
    data_ori["Fitxer"] = file
    data_ori["TipusPregunta"] = data_ori["Codi pregunta"].str[0]
    data_ori["TipusIncidencia"] = data_ori["Tipus incidencia"]

    # Obtain languages of comments    
    data_ori["Idiomas"] = data_ori.apply(lambda row: detect_language(row["Comentari"]), axis=1)
    data_ori["IdiomaSpacy"] = data_ori.apply(lambda row: row["Idiomas"][0], axis=1)
    data_ori["IdiomaPyCld2"] = data_ori.apply(lambda row: row["Idiomas"][1], axis=1)
    data_ori["Idioma"] = data_ori.apply(lambda row: row["Idiomas"][2], axis=1)

    data_res = data_ori[["Curs","Fitxer","TipusPregunta","TipusIncidencia","Comentari","IdiomaSpacy","IdiomaPyCld2","Idioma"]]
    #data_res = data_res.dropna(subset="Comentari")
    data_res = data_res[data_res["Comentari"].notnull()]
    
    if debug:
#        print("Comments:"  + data_res.shape[0])
        display(data_res[data_res["TipusIncidencia"].notnull()].head())
    
    return data_res

In [9]:
# Load data from file

def load_file(folder, file):
    try:
        url = folder + "/" + file
        if debug:
            print (Fore.BLACK + "File:   " + url)
        sheet = ""
        xlsx = pd.ExcelFile(url)
        for sheet in xlsx.sheet_names:
            if debug:
                print (Fore.BLACK + "Sheet:  " + sheet)
            data = pd.read_excel(xlsx, sheet)
            if validateSheet(data):
#                print ("Processing file: " + url)
                data_res = process_file(data, file)
                write_file(data_res)
    
    except:
        print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")

In [10]:
# Load data from folder

def load_folder(folder):
    
    if debug:
        print (Fore.BLACK + 'Folder: ' + folder)
    if os.path.isdir(folder):
        for file in os.listdir(folder):
            url = folder+"/"+file
#            print (url)
            if os.path.isdir(url):
                load_folder(url)
            if os.path.isfile(url):
                load_file(folder, file)


In [11]:
# Write data preprocessed into disk, in one file by language

def write_file(data_res):
    print ("write_file")
        
    if debug:
        print(data_res.value_counts("Idioma"))
    
    # Official languages
    languages = ["ca", "es", "en"]
    for language in languages:
        url = "comentaris_" + language + ".csv"
        data_lang = data_res[data_res["Idioma"]==language]
        
        if os.path.exists(pathdest + url):
            data_lang.to_csv(pathdest + url, index=False, mode="a", header=False)
            if debug:            
                print ("File updated: " + url)
        else:
            data_lang.to_csv(pathdest + url, index=False, mode="w", header=True)
            if debug:
                print ("File created: " + url)
            
    # Other languages        
    url = "comentaris_xx.csv"
    data_lang = data_res[~data_res["Idioma"].isin(languages)]
    if os.path.exists(pathdest + url):
        data_lang.to_csv(pathdest + url, index=False, mode="a", header=False)
        if debug:            
            print ("File updated: " + url)        
    else:
        data_lang.to_csv(pathdest + url, index=False, mode="w", header=True)
        if debug:
            print ("File created: " + url)
    
    if first_file:
        first_file = False
        

In [12]:
def analyze_files():
    
    if debug:
        print (Fore.BLACK + "Preprocessed file analysis:")

#   Load preprocessed files
    data_total = pd.DataFrame(columns=["Curs","TipusPregunta","TipusIncidencia","Idioma","File","Comentari"])

    languages = ["ca","es","en","xx"]
    for language in languages:
        file = "comentaris_" + language + ".csv"
        data_lang = pd.read_csv(pathdest + file)

        if debug:
            print ("* " + file + ": ", end="")
            print (data_lang.shape[0], end="")
            print (" registers")

        data_lang["Fitxer"] = language
        data_lang = data_lang.groupby(["Curs","TipusPregunta","TipusIncidencia","Idioma","Fitxer"], dropna=False).count().reset_index()
        data_total = pd.concat([data_total, data_lang])

    data_total = data_total.reset_index()
    display (data_total)

#   Distribution by Curs    
    data_group = data_total[["Curs","Fitxer","Comentari"]].groupby(["Curs","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="Curs", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)
        
#   Distribution by Tipus Pregunta        
    data_group = data_total[["TipusPregunta","Fitxer","Comentari"]].groupby(["TipusPregunta","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="TipusPregunta", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)
        
#   Distribution by Tipus Incidencia        
    data_group = data_total[["TipusIncidencia","Fitxer","Comentari"]].groupby(["TipusIncidencia","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="TipusIncidencia", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)

#   Distribution by Idioma        
    data_group = data_total[["Idioma","Fitxer","Comentari"]].groupby(["Idioma","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="Idioma", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)
        

In [13]:
# Detects the language of a comment. Compares results from Spacy and PyCld2

def detect_language(comment):
    
    lang_spacy = nlp(comment)._.language["language"]
    isReliable, textBytesFound, details = cld2.detect(comment.encode('utf-8', 'replace'), 
                                                    isPlainText = True, bestEffort = True, returnVectors=False)
    lang_pycld2 = details[0][1]
    
    language = lang_spacy
    if (lang_spacy != "ca") & (lang_pycld2 == "ca"):
        language = "ca"

    if (lang_spacy != "ca") & (lang_spacy != "es") & (lang_pycld2 == "es"):
        language = "es"
        
    if (lang_spacy == "en") & (lang_pycld2 != "ca") & (lang_pycld2 != "es") & (lang_pycld2 != "en"):
        language = "ca"
        
    if (debug == 2):
        print (comment, ", ", lang_spacy, ", ", lang_pycld2, ", ", language)
     
 #   language = lang_spacy
    
 #   return language
    return [lang_spacy, lang_pycld2, language]

In [14]:
# Main function

delete_preprocessed_files()
load_folder(pathori)
analyze_files()
print (Fore.BLACK + "ORIGINAL FILES PREPROCESSED")


Folder: ../data/original
Folder: ../data/original/17-18
File:   ../data/original/17-18/assignatura_professor_comentaris_LS_17-18_1s.xls
Sheet:  Hoja1
        VALID


,Curs,Fitxer,TipusPregunta,TipusIncidencia,Comentari,IdiomaSpacy,IdiomaPyCld2,Idioma
20,2017-18,assignatura_professor_comentaris_LS_17-18_1s.xls,P,Comentari d'assignatura,Módulo 1 áreas para mejorar: _x000D_\nLa tarea...,es,es,es
24,2017-18,assignatura_professor_comentaris_LS_17-18_1s.xls,P,No ha impartit classe a aquest grup,No he tenido contacto,es,es,es
25,2017-18,assignatura_professor_comentaris_LS_17-18_1s.xls,P,No ha impartit classe a aquest grup,No he tratado con él,pt,es,es
27,2017-18,assignatura_professor_comentaris_LS_17-18_1s.xls,P,No ha impartit classe a aquest grup,No he tratado con él,pt,es,es
29,2017-18,assignatura_professor_comentaris_LS_17-18_1s.xls,P,No ha impartit classe a aquest grup,No he tratado con él,pt,es,es


write_file
Idioma
es    52
ca    24
dtype: int64
File created: comentaris_ca.csv
File created: comentaris_es.csv
File created: comentaris_en.csv
File created: comentaris_xx.csv
        NO VALID
File:   ../data/original/17-18/assignatura_professor_comentaris_LS_17-18_1s_M110.xls
Sheet:  Sheet0
        VALID


,Curs,Fitxer,TipusPregunta,TipusIncidencia,Comentari,IdiomaSpacy,IdiomaPyCld2,Idioma
5,2017-18,assignatura_professor_comentaris_LS_17-18_1s_M...,P,Comentari d'assignatura,"No he après res, segueixo sense saber llatí.",ca,ca,ca


write_file
Idioma
es    10
ca     4
dtype: int64
File updated: comentaris_ca.csv
File updated: comentaris_es.csv
File updated: comentaris_en.csv
File updated: comentaris_xx.csv
        NO VALID
File:   ../data/original/17-18/assignatura_professor_comentaris_LS_17-18_2s.xls
Sheet:  Sheet0
        VALID
        NO VALID
File:   ../data/original/17-18/revisar.txt
        NO VALID
Folder: ../data/original/18-19
File:   ../data/original/18-19/assignatura_professor_comentaris_LS_18-19_P_1S.xls
Sheet:  Sheet0
        VALID


...

Fitxer,ca,en,es,xx
Curs,,,,
2017-18,28.0,NaN,62.0,NaN
2018-19,936.0,204.0,302.0,26.0
2019-20,5933.0,32.0,1401.0,66.0


Fitxer,ca,en,es,xx
TipusPregunta,,,,
A,2909.0,91.0,820.0,20.0
P,3988.0,145.0,945.0,72.0
ca,0.0,NaN,NaN,NaN
en,0.0,NaN,NaN,NaN


Fitxer,ca,en,es,xx
TipusIncidencia,,,,
Canvi de professor,16.0,NaN,2.0,NaN
Comentari d'assignatura,3.0,1.0,3.0,NaN
Comentari de professor,37.0,2.0,18.0,NaN
Comentari excel·lent,1.0,2.0,1.0,NaN
Comentari ofensiu,7.0,NaN,2.0,3.0
Comentari problemàtic,287.0,NaN,53.0,2.0
Duplicar a professor,2.0,NaN,3.0,NaN
Exclamacions o emoticones excessius,1.0,NaN,NaN,1.0
Faltes d'ortografia,14.0,NaN,4.0,NaN


Fitxer,ca,en,es,xx
Idioma,,,,
UNKNOWN,NaN,NaN,NaN,10.0
af,NaN,NaN,NaN,2.0
ca,6886.0,NaN,NaN,NaN
de,NaN,NaN,NaN,4.0
en,NaN,236.0,NaN,NaN
es,NaN,NaN,1765.0,NaN
fr,NaN,NaN,NaN,22.0
hr,NaN,NaN,NaN,1.0
id,NaN,NaN,NaN,2.0


ORIGINAL FILES PREPROCESSED
